# [GALAXY] - Drizzled Images Inpainter

<div class="alert alert-block alert-info">
    <b>Note:</b> This notebook should be run with the <span style="font-family: 'Ariel', monospace;">Astroba</span> environment.
</div>

Due to new DQ arrays and reference files. The new ACS images have many `NaN` holes in the image. This notebook is designed to implement variois filters over the image to fill in those holes.

## Prerequisites

Note that this notebook depends on a set of
[DS9](https://sites.google.com/cfa.harvard.edu/saoimageds9/home) region files to be present to run.
As we do not want to replace `NaN` pixels that are outside the field of view (FOV) of the images,
open the drizzled images generated by the previous notebook using DS9 and create a polygon around
the pixels you want replaced. Right now, this notebook assumes the FOV for each instrument is the
same. Therefore, the region file needs to be saved in sky coordinates as
`../DS9/FOVs/<INSTRUMENT>-<DETECTOR>-FOV.reg`.
For example, if defining the FOV for
[F475W](https://hst-docs.stsci.edu/wfc3ihb/appendix-a-wfc3-filter-throughputs/a-2-throughputs-and-signal-to-noise-ratio-data/uvis-f475w) &
[F814W](https://hst-docs.stsci.edu/wfc3ihb/appendix-a-wfc3-filter-throughputs/a-2-throughputs-and-signal-to-noise-ratio-data/uvis-f814w) which both utilize the
[ACS](https://hst-docs.stsci.edu/display/ACSDHB), the corresponding region file would be named
`ACS-WFC-FOV.reg`. For an example use case, consider checking out the [FOV region files defined for the
NGC 3568 data](https://github.com/wwaldron/ngc3568/tree/main/Images/ProcessedImages/HST/DS9/FOVs).

## Additional Notes

This notebook provides three method for replacing missing pixels. You may choose any subset of the
methods below.

* [Biharmonic Interpolator](#scikit-images-biharmonic-inpainter)
* [Median Filtering](#ndimages-generic-filter-with-numpys-nanmedian)
* [Convolutional Interpolator](#astropys-convolutional-interpolator)

## Imports

In [ ]:
# Python Imports
import os
from pathlib import Path
from glob import glob

# 3rd Party Imports
from tqdm.notebook import tqdm
import numpy as np
from scipy import ndimage as ndi
from skimage.restoration import inpaint_biharmonic as ibh
from llc import jit_filter_function as jff

# Astropy Imports
from astropy.io import fits
from astropy.wcs import WCS
from astropy.convolution import interpolate_replace_nans as irepnan, Gaussian2DKernel
from regions import Regions

## Notebook Setup

In [ ]:
# Numpy Setup
_ = np.seterr(all='ignore')

In [ ]:
# Get the Notebook's Directory and ChangeDir to it
NOTEBOOK_DIR = Path(__file__).resolve().parent
os.chdir(NOTEBOOK_DIR)

In [ ]:
# Directories
DRZ_DIR = Path('../Drizzled').resolve()
INP_DIR = Path('../Inpainted').resolve()
DS9_DIR = Path('../DS9/FOVs').resolve()

## Process Images

### SciKit-Image's [Biharmonic Inpainter](https://scikit-image.org/docs/stable/api/skimage.restoration.html#skimage.restoration.inpaint_biharmonic)

In [ ]:
%%bash
# Make Output Directory
mkdir -p ../Inpainted

In [ ]:
# Loop through Images
for fileName in tqdm(DRZ_DIR.glob('*_drc.fits')):

    # Get Output Name
    outName = INP_DIR / fileName.name.replace('.fits', '_ibh.fits')

    # Open the File for Saving
    with fits.open(fileName) as hduList:

        # Get Instrument
        instr = hduList[0].header['PRIMESI'].strip()
        det   = hduList[0].header['DETECTOR'].strip()

        # Extract Image
        img = hduList[1].data

        # Load Regions
        regs = Regions.read(DS9_DIR / f'{instr}-{det}-FOV.reg', format='ds9')
        fov  = regs[0].to_pixel(WCS(hduList[1]))
        fovMsk = fov.to_mask().to_image(img.shape).astype(bool)

        # Mask of Pixels to Replace
        # Replace Pixels in the FOV that are NaN
        repPixMsk = fovMsk & np.isnan(img)

        # New Image with NaNs replaced with zeros
        zeroedImg = img.copy()
        zeroedImg[np.isnan(img)] = 0

        # Filter the Image
        filtImg = ibh(
            zeroedImg,
            mask=repPixMsk
        )

        # Re-NaN the Border
        filtImg[~fovMsk & np.isnan(img)] = np.NaN

        # Store to New Image
        outList = hduList.copy()
        outList[0].header.add_history('NaN pixels in FOV replaced with Biharmonic Inpainting')
        outList[0].header.add_comment('See https://scikit-image.org/docs/stable/api/skimage.restoration.html')
        outList['SCI'].data = filtImg
        outList.writeto(outName, overwrite=True)

### NDImage's [Generic Filter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.generic_filter.html) with Numpy's [nanmedian](https://numpy.org/doc/stable/reference/generated/numpy.nanmedian.html)

In [ ]:
@jff
def llcnanmed(x):
    result = np.nanmedian(x)
    return result

In [ ]:
# Loop through Images
for fileName in tqdm(DRZ_DIR.glob('*_drc.fits')):

    # Get Output Name
    outName = INP_DIR / fileName.name.replace('.fits', '_med.fits')

    # Open the File for Saving
    with fits.open(fileName) as hduList:

        # Get Instrument
        instr = hduList[0].header['PRIMESI'].strip()
        det   = hduList[0].header['DETECTOR'].strip()

        # Extract Image
        img = hduList[1].data

        # Load Regions
        regs = Regions.read(DS9_DIR / f'{instr}-{det}-FOV.reg', format='ds9')
        fov  = regs[0].to_pixel(WCS(hduList[1]))
        fovMsk = fov.to_mask().to_image(img.shape).astype(bool)

        # Mask of Pixels to Replace
        # Replace Pixels in the FOV that are NaN
        repPixMsk = fovMsk & np.isnan(img)

        # Do the Median Filtering
        filtImg = ndi.generic_filter(
            img, llcnanmed, size=5
        )

        # Replace NaN pixels in the FOV
        img[repPixMsk] = filtImg[repPixMsk]

        # Store to New Image
        outList = hduList.copy()
        outList[0].header.add_history('NaN pixels in FOV replaced with NaN-Median Filtering')
        outList['SCI'].data = img
        outList.writeto(outName, overwrite=True)

### Astropy's Convolutional Interpolator

In [ ]:
# Kernel
KERN = Gaussian2DKernel(1.0)

# Loop through Images
for fileName in tqdm(DRZ_DIR.glob('*_drc.fits')):

    # Get Output Name
    outName = INP_DIR / fileName.name.replace('.fits', '_irn.fits')

    # Open the File for Saving
    with fits.open(fileName) as hduList:

        # Get Instrument
        instr = hduList[0].header['PRIMESI'].strip()
        det   = hduList[0].header['DETECTOR'].strip()

        # Extract Image
        img = hduList[1].data

        # Load Regions
        regs = Regions.read(DS9_DIR / f'{instr}-{det}-FOV.reg', format='ds9')
        fov  = regs[0].to_pixel(WCS(hduList[1]))
        fovMsk = fov.to_mask().to_image(img.shape).astype(bool)

        # Mask of Pixels to Replace
        # Replace Pixels in the FOV that are NaN
        repPixMsk = fovMsk & np.isnan(img)

        # Do the Median Filtering
        filtImg = irepnan(img, KERN)

        # Replace NaN pixels in the FOV
        img[repPixMsk] = filtImg[repPixMsk]

        # Store to New Image
        outList = hduList.copy()
        outList[0].header.add_history("NaN pixels in FOV replaced with Astropy's Convolutional Interpolator")
        outList[0].header.add_comment('See https://docs.astropy.org/en/stable/api/astropy.convolution.interpolate_replace_nans.html')
        outList['SCI'].data = img
        outList.writeto(outName, overwrite=True)